## Day4-3 : 把空氣品質資料寫入DB中(sqlite3)

In [15]:
import requests

In [16]:
url = 'http://opendata.epa.gov.tw/ws/Data/AQI/?$format=json'

In [17]:
r = requests.get(url)

In [18]:
r

<Response [200]>

In [19]:
data = r.json()

In [6]:
len(data)

81

In [7]:
data[0]

{'SiteName': '臺南(北門)',
 'County': '臺南市',
 'AQI': '15',
 'Pollutant': '',
 'Status': '良好',
 'SO2': '0.2',
 'CO': '0.07',
 'CO_8hr': '0.1',
 'O3': '20',
 'O3_8hr': '16',
 'PM10': '4',
 'PM2.5': 'ND',
 'NO2': '0',
 'NOx': '0.3',
 'NO': '0.4',
 'WindSpeed': '1.8',
 'WindDirec': '277',
 'PublishTime': '2020-07-25 13:00',
 'PM2.5_AVG': '4',
 'PM10_AVG': '11',
 'SO2_AVG': '0',
 'Longitude': '120.124444',
 'Latitude': '23.264722',
 'SiteId': '310'}

- PublishTime, County, SiteName, AQI, PM10

In [20]:
for item in data:
    if item['County'] == '高雄市':
        print('[%s %s][%s] %s, %s' %(item['County'], item['SiteName'], item['PublishTime'], item['AQI'], item['PM10']) )

[高雄市 復興][2020-07-25 13:00] 16, 8
[高雄市 小港][2020-07-25 13:00] 15, 8
[高雄市 前鎮][2020-07-25 13:00] 14, 17
[高雄市 前金][2020-07-25 13:00] 18, 8
[高雄市 左營][2020-07-25 13:00] 15, 7
[高雄市 楠梓][2020-07-25 13:00] 14, 7
[高雄市 林園][2020-07-25 13:00] 19, 7
[高雄市 大寮][2020-07-25 13:00] 18, 18
[高雄市 鳳山][2020-07-25 13:00] 17, 11
[高雄市 仁武][2020-07-25 13:00] 16, 9
[高雄市 橋頭][2020-07-25 13:00] 16, 5
[高雄市 美濃][2020-07-25 13:00] 18, 13


### 存入DB, 再讀回來

In [11]:
import sqlite3
import time
import json

In [12]:
# initial local DB
# PublishTime, County, SiteName, AQI, PM10
aqi_db = 'aqi_db.db'
conn = sqlite3.connect(aqi_db)
cursor = conn.cursor()
cursor.execute('create table IF NOT EXISTS aqi (PublishTime text NOT NULL, County text, SiteName text, AQI int, PM10 int)')
cursor.close()
conn.commit()
conn.close()

In [13]:
for item in data:
    if item['County'] == '高雄市':
        print('[%s %s][%s] %s, %s' %(item['County'], item['SiteName'], item['PublishTime'], item['AQI'], item['PM10']) )
        
        #- insert into local DB
        conn = sqlite3.connect(aqi_db)
        cursor = conn.cursor()
        cursor.execute("insert into aqi (PublishTime, County, SiteName, AQI, PM10) values('%s', '%s', '%s', '%s', '%s')" 
                       %(item['PublishTime'], item['County'], item['SiteName'], item['AQI'], item['PM10']))
        cursor.close()
        conn.commit()
        conn.close()        

[高雄市 復興][2020-07-25 13:00] 16, 8
[高雄市 小港][2020-07-25 13:00] 15, 8
[高雄市 前鎮][2020-07-25 13:00] 14, 17
[高雄市 前金][2020-07-25 13:00] 18, 8
[高雄市 左營][2020-07-25 13:00] 15, 7
[高雄市 楠梓][2020-07-25 13:00] 14, 7
[高雄市 林園][2020-07-25 13:00] 19, 7
[高雄市 大寮][2020-07-25 13:00] 18, 18
[高雄市 鳳山][2020-07-25 13:00] 17, 11
[高雄市 仁武][2020-07-25 13:00] 16, 9
[高雄市 橋頭][2020-07-25 13:00] 16, 5
[高雄市 美濃][2020-07-25 13:00] 18, 13


In [14]:
# 取出全部

conn = sqlite3.connect(aqi_db)
cursor = conn.cursor()
# select last 5 rows
res = cursor.execute('SELECT * FROM aqi')
rows = res.fetchall()

data_list = list(rows)
data_list

[('2020-07-25 13:00', '高雄市', '復興', 16, 8),
 ('2020-07-25 13:00', '高雄市', '小港', 15, 8),
 ('2020-07-25 13:00', '高雄市', '前鎮', 14, 17),
 ('2020-07-25 13:00', '高雄市', '前金', 18, 8),
 ('2020-07-25 13:00', '高雄市', '左營', 15, 7),
 ('2020-07-25 13:00', '高雄市', '楠梓', 14, 7),
 ('2020-07-25 13:00', '高雄市', '林園', 19, 7),
 ('2020-07-25 13:00', '高雄市', '大寮', 18, 18),
 ('2020-07-25 13:00', '高雄市', '鳳山', 17, 11),
 ('2020-07-25 13:00', '高雄市', '仁武', 16, 9),
 ('2020-07-25 13:00', '高雄市', '橋頭', 16, 5),
 ('2020-07-25 13:00', '高雄市', '美濃', 18, 13)]

- 整合成排程

In [21]:
import requests
import sqlite3
import time
import json

url = 'http://opendata.epa.gov.tw/ws/Data/AQI/?$format=json'
r = requests.get(url)
data = r.json()

# initial local DB
aqi_db = 'aqi_db.db'
conn = sqlite3.connect(aqi_db)
cursor = conn.cursor()
cursor.execute('create table IF NOT EXISTS aqi (PublishTime text NOT NULL, County text, SiteName text, AQI int, PM10 int)')
cursor.close()
conn.commit()
conn.close()

for item in data:
    if item['County'] == '高雄市':
        print('[%s %s][%s] %s, %s' %(item['County'], item['SiteName'], item['PublishTime'], item['AQI'], item['PM10']) )
        
        #- insert into local DB
        conn = sqlite3.connect(aqi_db)
        cursor = conn.cursor()
        cursor.execute("insert into aqi (PublishTime, County, SiteName, AQI, PM10) values('%s', '%s', '%s', '%s', '%s')" 
                       %(item['PublishTime'], item['County'], item['SiteName'], item['AQI'], item['PM10']))
        cursor.close()
        conn.commit()
        conn.close()

[高雄市 復興][2020-07-25 13:00] 16, 8
[高雄市 小港][2020-07-25 13:00] 15, 8
[高雄市 前鎮][2020-07-25 13:00] 14, 17
[高雄市 前金][2020-07-25 13:00] 18, 8
[高雄市 左營][2020-07-25 13:00] 15, 7
[高雄市 楠梓][2020-07-25 13:00] 14, 7
[高雄市 林園][2020-07-25 13:00] 19, 7
[高雄市 大寮][2020-07-25 13:00] 18, 18
[高雄市 鳳山][2020-07-25 13:00] 17, 11
[高雄市 仁武][2020-07-25 13:00] 16, 9
[高雄市 橋頭][2020-07-25 13:00] 16, 5
[高雄市 美濃][2020-07-25 13:00] 18, 13


In [22]:
# 取出全部

conn = sqlite3.connect(aqi_db)
cursor = conn.cursor()
# select last 5 rows
res = cursor.execute('SELECT * FROM aqi')
rows = res.fetchall()

data_list = list(rows)
data_list

[('2020-07-25 13:00', '高雄市', '復興', 16, 8),
 ('2020-07-25 13:00', '高雄市', '小港', 15, 8),
 ('2020-07-25 13:00', '高雄市', '前鎮', 14, 17),
 ('2020-07-25 13:00', '高雄市', '前金', 18, 8),
 ('2020-07-25 13:00', '高雄市', '左營', 15, 7),
 ('2020-07-25 13:00', '高雄市', '楠梓', 14, 7),
 ('2020-07-25 13:00', '高雄市', '林園', 19, 7),
 ('2020-07-25 13:00', '高雄市', '大寮', 18, 18),
 ('2020-07-25 13:00', '高雄市', '鳳山', 17, 11),
 ('2020-07-25 13:00', '高雄市', '仁武', 16, 9),
 ('2020-07-25 13:00', '高雄市', '橋頭', 16, 5),
 ('2020-07-25 13:00', '高雄市', '美濃', 18, 13),
 ('2020-07-25 13:00', '高雄市', '復興', 16, 8),
 ('2020-07-25 13:00', '高雄市', '小港', 15, 8),
 ('2020-07-25 13:00', '高雄市', '前鎮', 14, 17),
 ('2020-07-25 13:00', '高雄市', '前金', 18, 8),
 ('2020-07-25 13:00', '高雄市', '左營', 15, 7),
 ('2020-07-25 13:00', '高雄市', '楠梓', 14, 7),
 ('2020-07-25 13:00', '高雄市', '林園', 19, 7),
 ('2020-07-25 13:00', '高雄市', '大寮', 18, 18),
 ('2020-07-25 13:00', '高雄市', '鳳山', 17, 11),
 ('2020-07-25 13:00', '高雄市', '仁武', 16, 9),
 ('2020-07-25 13:00', '高雄市', '橋頭', 16, 5),
 ('2

### 練習題: 放到GCE排程，定時10分鐘收集一次資料

### 完成WEB API: 把一天的資料都以JSON的格式回傳

